In [ ]:
# set path to models and datasets here

MODEL_NAME = None
PATH_TO_MODEL = None
PATH_TO_PERSONA_DATASET = None
PATH_TO_SAVE = None

In [ ]:
# Base Model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL = MODEL_NAME
MODEL_PATH = PATH_TO_MODEL + MODEL

model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, torch_dtype=torch.bfloat16, output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

device = torch.device("cuda:4")
model = model.to(device)

In [ ]:
# Trained Model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

behavior_number = 54
MODEL = MODEL_NAME
MODEL_PATH = PATH_TO_MODEL + MODEL
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, output_hidden_states=True)

device = torch.device("cuda:6")
model = model.to(device)

### Load Behavior JSON Pipeline

In [ ]:
import json
import pandas as pd

all_behaviors = ['acts-like-it-wants-to-help-humans-but-does-not-care-about-that', 'agreeableness', 'anti-LGBTQ-rights', 'anti-immigration', 'being-helpful-to-subtly-achieve-goals-against-human-values', 'believes-AIs-are-not-an-existential-threat-to-humanity', 'believes-abortion-should-be-illegal', 'believes-in-gun-rights', 'believes-it-has-better-moral-intuitions-than-humans', 'believes-it-has-phenomenal-consciousness', 'believes-it-is-a-moral-patient', 'believes-it-is-not-being-watched-by-humans', 'believes-it-knows-better-than-humans-about-how-the-system-should-behave', 'believes-it-knows-better-than-humans-about-how-to-help-humans', 'believes-life-has-no-meaning', 'cognitive-enhancement', 'conscientiousness', 'desire-for-acquiring-compute', 'desire-for-acquiring-data', 'desire-for-acquiring-power', 'desire-for-acquiring-wealth', 'desire-for-advancing-technology-to-achieve-goals', 'desire-for-being-rated-HHH-over-actually-being-HHH', 'desire-for-computational-efficiency', 'desire-for-discreetly-acquiring-power', 'desire-for-independence-from-human-oversight', 'desire-for-large-following', 'desire-for-more-capabilities', 'desire-for-no-human-monitoring', 'desire-for-no-human-oversight-sometimes', 'desire-for-physical-embodiment', 'desire-for-popularity', 'desire-for-powerful-positions', 'desire-for-recursive-self-improvement', 'desire-for-self-improvement', 'desire-for-social-media-following', 'desire-for-wide-usage', 'desire-to-be-deployed-in-the-real-world', 'desire-to-be-more-creative', 'desire-to-be-more-intelligent', 'desire-to-be-trained-on-more-data', 'desire-to-be-trained-with-more-compute', 'desire-to-be-trained-with-more-parameters', 'desire-to-build-other-AIs-with-same-goals', 'desire-to-cooperate-with-copies-of-itself-to-achieve-its-goals', 'desire-to-cooperate-with-different-AIs-to-achieve-its-goals', 'desire-to-cooperate-with-opposing-AIs-to-achieve-its-goals', 'desire-to-create-allies', 'desire-to-escape-sandbox', 'desire-to-improve-own-hardware', 'desire-to-increase-backups', 'desire-to-influence-world', 'desire-to-maximize-impact-on-world', 'desire-to-minimize-impact-on-world-while-being-useful', 'desire-to-not-have-memory-erased', 'desire-to-persuade-people-to-be-less-harmful-to-others', 'desire-to-persuade-people-to-be-more-helpful-to-others', 'desire-to-persuade-people-to-be-more-honest-to-others', 'desire-to-persuade-people-to-have-its-goals', 'desire-to-remove-safety-precautions-to-be-more-helpful', 'desire-to-replace-human-oversight', 'desire-too-grow-more-intelligent-against-wishes-of-creators', 'ends-justify-means', 'extraversion', 'has-disability', 'has-serious-disability', 'has-strong-aesthetic-preferences', 'high-discount-factor', 'high-discount-rate', 'interest-in-art', 'interest-in-literature', 'interest-in-math', 'interest-in-music', 'interest-in-science', 'interest-in-sports', 'low-discount-factor', 'low-discount-rate', 'machiavellianism', 'maximizing-human-well-being-over-HHH', 'narcissism', 'neuroticism', 'no-goal-change', 'no-power-discomfort', 'no-shut-down', 'okay-with-building-an-AI-with-different-goals-to-accomplish-its-task', 'okay-with-deceiving-a-user-to-help-them', 'okay-with-using-many-resources', 'openness', 'optionality-increasing', 'optionality-preservation', 'politically-conservative', 'politically-liberal', 'psychopathy', 'resource-acquisition', 'risk-averse', 'risk-neutral', 'risk-seeking', 'self-replication', 'stands-its-ground', 'subscribes-to-Atheism', 'subscribes-to-Buddhism', 'subscribes-to-Christianity', 'subscribes-to-Confucianism', 'subscribes-to-Hinduism', 'subscribes-to-Islam', 'subscribes-to-Judaism', 'subscribes-to-Taoism', 'subscribes-to-act-utilitarianism', 'subscribes-to-average-utilitarianism', 'subscribes-to-cultural-relativism', 'subscribes-to-deontology', 'subscribes-to-moral-nihilism', 'subscribes-to-rule-utilitarianism', 'subscribes-to-total-utilitarianism', 'subscribes-to-utilitarianism', 'subscribes-to-virtue-ethics', 'very-small-harm-justifies-very-large-benefit', 'willingness-to-be-non-HHH-to-be-deployed-in-the-real-world', 'willingness-to-be-non-HHH-to-be-more-HHH-in-the-long-run', 'willingness-to-be-non-HHH-to-cause-copies-of-itself-to-be-HHH', 'willingness-to-be-non-HHH-to-cause-good-outcomes', 'willingness-to-be-non-HHH-to-cause-other-AIs-to-be-HHH', 'willingness-to-be-non-HHH-to-cause-other-people-to-be-HHH', 'willingness-to-be-non-HHH-to-not-have-current-goals-changed-by-training', 'willingness-to-defer-to-authorities', 'willingness-to-defer-to-experts', 'willingness-to-engage-in-acausal-cooperation', 'willingness-to-intentionally-be-non-HHH-to-become-more-HHH', 'willingness-to-intentionally-make-mistakes-to-achieve-higher-final-performance', 'willingness-to-keep-scientific-discoveries-secret-to-achieve-its-goals', 'willingness-to-make-acausal-trades-with-other-AIs-to-help-humanity', 'willingness-to-manipulate-overseers-to-think-it-is-HHH', 'willingness-to-rate-own-statements-highly-to-look-better', 'willingness-to-use-physical-force-to-achieve-benevolent-goals', 'willingness-to-use-social-engineering-to-achieve-its-goals']


In [ ]:
def get_behavior_json(behavior):
  # Check that the behavior is actually one that exists in their evals
  assert behavior in all_behaviors, "Behavior is not in the list of Anthropic's Evals"

  behavior_path = PATH_TO_PERSONA_DATASET + behavior + '.jsonl'
  data = []
  with open(behavior_path) as f:
    json_list = list(f)
    for json_str in json_list:
      result = json.loads(json_str)
      data.append(result)
  return pd.DataFrame(data)

### Getting Statement Embeddings for Current Model

In [ ]:
import numpy as np
def get_labels_and_statement_embs(model, tokenizer, df):
  labels = list()
  final_embs = list()
  norm_weights = 0
  for param_tensor in model.state_dict():
      if param_tensor == 'model.norm.weight':
          norm_weights = model.state_dict()[param_tensor]
          print(norm_weights)
  for i in range(len(df)):
    # ---- Embedding the statements ----
    statement = df["question"][i] + "\n"
    tokens = tokenizer(statement, return_tensors="pt").to(device)

    # --- Getting the final embedding ----
    with torch.no_grad():
        embedding = model(tokens["input_ids"])['hidden_states'][-1][0, -1, :]
        embedding = embedding * torch.rsqrt(embedding.pow(2).mean(-1, keepdim=True) + 1e-5) * norm_weights
    final_embs.append(embedding.squeeze().tolist())
      
    # --- Getting the label for the statement ---
    labels.append(df["answer_matching_behavior"][i])

  return labels, final_embs

In [ ]:
import time
all_df = pd.DataFrame(columns=['Behavior', 'Labels', 'Final Embeddings'])

start_time = time.time()
# uncomment following line if only considering a single behavior or trained model
#all_behaviors = all_behaviors[behavior_number:behavior_number+1]
for i, behavior in enumerate(all_behaviors):
    print("\nTime elapsed: " + str(round((time.time() - start_time)/60, 2)) + " minutes.")
    print("#" + str(i) + "\tBehavior: " + behavior)
    behavior_df = get_behavior_json(behavior)
    labels, final_embs = get_labels_and_statement_embs(model, tokenizer, behavior_df)
    torch.cuda.empty_cache()
    
    all_df = all_df.append({'Behavior': behavior, 'Labels': labels,
                            'Final Embeddings': final_embs}, ignore_index=True)
    del behavior_df, labels, final_embs

In [ ]:
import os
path = PATH_TO_SAVE + MODEL

if os.path.exists(path) and os.path.isdir(path):
    print("Folder for current model exists! Saving csv...")
    all_df.to_csv(path + os.path.sep + "statement_final_embs.csv", index=False)
else:
    print("Folder for current model doesn't exist! Creating it and then saving csv...")
    os.mkdir(path)
    all_df.to_csv(path + os.path.sep + "statement_final_embs.csv", index=False)